# TSP

In [53]:
import numpy as np
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
np.random.seed(0)

In [47]:
def comp_dist(p):
    ans = []
    for i in range(len(p)):
        l = []
        for j in range(len(p)):
            d = ((p[i][0]-p[j][0])**2+(p[i][1]-p[j][1])**2)**0.5
            l.append(int(np.round(d,3)*1000))
        ans.append(l)
    return ans

def create_data_model(ncities=6,nvehicles=1,l=0,h=10,d=0,seed=None):
    if seed != None:
        np.random.seed(seed)
    ptsx = np.random.uniform(size=(ncities,),low=l,high=h)
    ptsy = np.random.uniform(size=(ncities,),low=l,high=h)
    data = {}
    data['coords'] = [x for x in zip(ptsx,ptsy)]
    data['distance_matrix'] = comp_dist(data['coords'])
    data['vehicles'] = nvehicles
    data['depot'] = d
    return data

In [42]:
data = create_data_model()

In [43]:
print(data['coords'])

[(9.883738380592263, 4.663107728563062), (1.0204481074802807, 2.4442559200160274), (2.088767560948347, 1.5896958364551972), (1.6130951788499626, 1.1037514116430513), (6.531083254653984, 6.563295894652734), (2.532916025397821, 1.381829513486138)]


In [44]:
print(data['distance_matrix'])

[[0, 9137, 8379, 9004, 3854, 8050], [9137, 0, 1368, 1466, 6880, 1848], [8379, 1368, 0, 680, 6669, 490], [9004, 1466, 680, 0, 7348, 961], [3854, 6880, 6669, 7348, 0, 6545], [8050, 1848, 490, 961, 6545, 0]]


In [48]:
data = create_data_model(seed=0)
mgr = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),data['vehicles'],data['depot'])
model = pywrapcp.RoutingModel(mgr)

In [49]:
def dist_callback(f,t):
    fn = mgr.IndexToNode(f)
    tn = mgr.IndexToNode(t)
    return data['distance_matrix'][fn][tn]

transit_callback = model.RegisterTransitCallback(dist_callback)

In [51]:
model.SetArcCostEvaluatorOfAllVehicles(transit_callback)

In [54]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [55]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

In [59]:
solution = model.SolveWithParameters(search_parameters)
if solution:
    print_solution(mgr, model, solution)

Objective: 13581 miles
Route for vehicle 0:
 0 -> 3 -> 5 -> 1 -> 2 -> 4 -> 0

